In [7]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial import cKDTree
import time

# ======================
# CONFIG
# ======================
CHARGER_FILE = 'ev_chargers_consolidated_sep25.csv'
REGISTRATION_FILE = 'vehicles registration.csv'
POA_SHAPEFILE = 'POA_2021_AUST_GDA2020.shp'
TRAFFIC_SENSORS_CSV = 'Traffic_Volume_Viewer_2025_Data.csv'
POP_CSV = "2021Census_G04B_NSW_POA.csv"
TRAVEL_CSV = "2021Census_G62_NSW_POA.csv"

OUT_DIR = 'output/stages'
os.makedirs(OUT_DIR, exist_ok=True)

ZERO_PLUG_PROXY_RATIO = 99999
CHARGER_BUFFER_M = 5000  # 5 km
SENSOR_IDW_RADIUS_M = 20000  # for IDW fallback

# ======================
# UTILITY FUNCTIONS
# ======================
def print_header(msg):
    print("\n" + "="*6 + f" {msg} " + "="*6)

def ensure_pcode_str(df, col='pcode'):
    df[col] = df[col].astype(str).str.split('.').str[0].str.zfill(4)
    return df

def show_df_diag(df, label):
    print_header(label)
    print(df.head(5).to_string(index=False))
    print(f"\nShape: {df.shape}\n")

# ======================
# STAGE 1: EV DEMAND
# ======================
def stage_1_ev_demand():
    print_header("Stage 1: EV Registration (Demand)")
    df = pd.read_csv(REGISTRATION_FILE)
    df.columns = df.columns.str.lower().str.strip()

    df = ensure_pcode_str(df, 'registered_postcode' if 'registered_postcode' in df.columns else 'pcode')
    df = df.rename(columns={'registered_postcode': 'pcode'})

    df['no_vehicles'] = pd.to_numeric(df['no_vehicles'], errors='coerce').fillna(0)

    ev_mask = df['motive_power'].str.lower().str.strip() == "battery/fuel-cell electric"
    df_ev = df[ev_mask].groupby('pcode')['no_vehicles'].sum().reset_index().rename(columns={'no_vehicles':'ev_count'})
    df_total = df.groupby('pcode')['no_vehicles'].sum().reset_index().rename(columns={'no_vehicles':'total_vehicles'})

    df_demand = df_total.merge(df_ev, on='pcode', how='left')
    df_demand['ev_count'] = df_demand['ev_count'].fillna(0).astype(int)
    df_demand['ev_proportion'] = np.where(df_demand['total_vehicles']>0,
                                          df_demand['ev_count']/df_demand['total_vehicles'], 0)

    df_demand.to_csv(f"{OUT_DIR}/01_ev_demand.csv", index=False)
    show_df_diag(df_demand, "Saved: 01_ev_demand.csv")
    return df_demand

# ======================
# STAGE 2: CHARGER SUPPLY
# ======================
def stage_2_charger_supply():
    print_header("Stage 2: Charger Supply")
    df = pd.read_csv(CHARGER_FILE)
    df.columns = df.columns.str.lower().str.strip()

    df = ensure_pcode_str(df, 'pcode')
    df['num_plugs'] = pd.to_numeric(df['number_of_plugs'], errors='coerce').fillna(0).astype(int)

    df_active = df[~df['operator'].str.contains("upcoming", case=False, na=False)].copy()

    df_agg = df_active.groupby('pcode').agg(
        total_plugs=('num_plugs','sum'),
        total_stations=('number_of_stations','sum')
    ).reset_index()

    df_agg.to_csv(f"{OUT_DIR}/02_chargers_agg.csv", index=False)
    df_active[['pcode','latitude','longitude','num_plugs','operator']].to_csv(f"{OUT_DIR}/02_chargers_active_sample.csv", index=False)

    show_df_diag(df_agg, "Saved: 02_chargers_agg.csv")
    return df_agg, df_active

# ======================
# STAGE 3: MERGE INTO POA
# ======================
def stage_3_merge_poa(df_demand, df_ch_agg, poa_shapefile=POA_SHAPEFILE):
    print_header("STAGE 3 — Merge demand & supply into POA polygons")
    
    # Load POA polygons
    gdf = gpd.read_file(poa_shapefile)

    # Detect POA code column
    join_col = None
    for c in gdf.columns:
        if 'POA_CODE' in c.upper():
            join_col = c
            break
    if join_col is None:
        raise RuntimeError("POA shapefile: Can't locate a POA_CODE / POA_CODE21 column.")

    # Normalize join column & demand/supply keys
    gdf[join_col] = gdf[join_col].astype(str).str.zfill(4)
    df_demand['pcode'] = df_demand['pcode'].astype(str).str.zfill(4)
    df_ch_agg['pcode'] = df_ch_agg['pcode'].astype(str).str.zfill(4)

    # Merge demand and supply
    gdf = gdf.merge(df_demand, left_on=join_col, right_on='pcode', how='left')
    gdf = gdf.merge(df_ch_agg, on='pcode', how='left')

    # Fill missing values
    gdf['total_plugs'] = gdf['total_plugs'].fillna(0).astype(int)
    gdf['total_stations'] = gdf['total_stations'].fillna(0).astype(int)
    gdf['ev_count'] = gdf['ev_count'].fillna(0).astype(int)
    gdf['total_vehicles'] = gdf['total_vehicles'].fillna(0).astype(int)
    gdf['ev_proportion'] = gdf['ev_proportion'].fillna(0.0)

    # ---- Correct centroid computation (no CRS warnings) ----
    gdf_proj = gdf.to_crs(epsg=3857)  # project to meters first
    centroids = gdf_proj.geometry.centroid.to_crs(4326)  # convert back to lon/lat

    gdf['centroid_lon'] = centroids.x
    gdf['centroid_lat'] = centroids.y

    # Compute area (km²)
    gdf_proj = gdf.to_crs(epsg=3857)
    gdf['area_km2'] = gdf_proj.geometry.area / 1e6

    # Compute EV-to-charger ratio & flags
    gdf['ev_to_charger_ratio_raw'] = np.where(
        gdf['total_plugs'] == 0,
        np.where(gdf['ev_count'] > 0, ZERO_PLUG_PROXY_RATIO, 0),
        gdf['ev_count'] / gdf['total_plugs'].replace(0, np.nan)
    )
    gdf['ev_to_charger_ratio'] = gdf['ev_to_charger_ratio_raw'].replace(ZERO_PLUG_PROXY_RATIO, np.nan)
    gdf['has_plugs'] = (gdf['total_plugs'] > 0).astype(int)
    gdf['underserved_flag'] = ((gdf['ev_to_charger_ratio_raw'] == ZERO_PLUG_PROXY_RATIO) & (gdf['ev_count'] > 0)).astype(int)

    # Rename POA_CODE21 to POA_CODE if needed
    if join_col != 'POA_CODE':
        gdf = gdf.rename(columns={join_col: 'POA_CODE'})
    if 'POA_NAME' not in gdf.columns:
        gdf['POA_NAME'] = gdf['POA_CODE']

    # Save stage output
    out_csv = os.path.join(OUT_DIR, '03_poa_merged_attributes.csv')
    gdf.drop(columns='geometry').to_csv(out_csv, index=False)

    out_gpkg = os.path.join(OUT_DIR, '03_poa_merged.gpkg')
    try:
        gdf.to_file(out_gpkg, layer='poa_merged', driver='GPKG')
    except Exception as e:
        print("Warning: Could not save GeoPackage:", e)

    show_df_diag(gdf.drop(columns='geometry'), "POA merged attributes (saved)")
    return gdf

# ======================
# STAGE 4: NEARBY CHARGERS (BUFFER)
# ======================
def stage_4_nearby_chargers(gdf_poa, df_chargers_active, buffer_m=CHARGER_BUFFER_M):
    print_header(f"STAGE 4 — Nearby Chargers (within {buffer_m/1000:.1f} km)")

    # Ensure charger points are usable
    if not {'latitude','longitude','num_plugs'}.issubset(df_chargers_active.columns):
        print("❌ Active charger data missing lat/lon/num_plugs → skipping Stage 4.")
        gdf_poa[f'nearby_plugs_{int(buffer_m/1000)}km'] = 0
        gdf_poa[f'nearby_stations_{int(buffer_m/1000)}km'] = 0
        return gdf_poa

    # ---- Convert chargers to GeoDataFrame ----
    ch = gpd.GeoDataFrame(
        df_chargers_active.copy(),
        geometry=gpd.points_from_xy(df_chargers_active['longitude'], df_chargers_active['latitude']),
        crs='EPSG:4326'
    ).to_crs(3857)

    # ---- Convert POA polygons to projected CRS ----
    gdf = gdf_poa.copy()
    gdf = gdf[gdf.geometry.notnull() & ~gdf.geometry.is_empty]  # <— ✅ Skip invalid geometry rows
    gdf = gdf.to_crs(3857)

    # Spatial index
    ch_index = ch.sindex

    plugs_out = []
    stations_out = []

    print(f"Calculating nearby chargers for {len(gdf):,} POAs...")

    for idx, poly in enumerate(gdf.geometry):
        # Expand polygon to buffer radius
        search_area = poly.buffer(buffer_m)

        # Spatial index bounding box search first (fast)
        possible = list(ch_index.intersection(search_area.bounds))
        if not possible:
            plugs_out.append(0)
            stations_out.append(0)
            continue

        candidates = ch.iloc[possible]

        # Filter strictly within the distance radius
        dists = candidates.geometry.distance(poly.centroid)
        within = candidates[dists <= buffer_m]

        plugs_out.append(int(within['num_plugs'].sum()))
        stations_out.append(len(within))

        if idx % 50 == 0 and idx > 0:
            print(f"  processed {idx}/{len(gdf)} POAs...")

    # Write results back (aligned by index)
    out_col_plugs = f'nearby_plugs_{int(buffer_m/1000)}km'
    out_col_stations = f'nearby_stations_{int(buffer_m/1000)}km'

    gdf[out_col_plugs] = plugs_out
    gdf[out_col_stations] = stations_out

    # Put back into original CRS
    gdf = gdf.to_crs(4326)

    # Merge results back into original table (preserves rows previously skipped)
    gdf_poa = gdf_poa.merge(
        gdf[['POA_CODE', out_col_plugs, out_col_stations]],
        on='POA_CODE',
        how='left'
    )

    gdf_poa[out_col_plugs] = gdf_poa[out_col_plugs].fillna(0).astype(int)
    gdf_poa[out_col_stations] = gdf_poa[out_col_stations].fillna(0).astype(int)

    # Save stage output
    out_csv = os.path.join(OUT_DIR, f"04_nearby_chargers_{int(buffer_m/1000)}km.csv")
    gdf_poa.drop(columns='geometry').to_csv(out_csv, index=False)
    print(f"✅ Nearby-charger aggregates saved to {out_csv}")

    return gdf_poa

# ======================
# STAGE 5: TRAFFIC + IDW
# ======================
def stage_5_traffic(gdf):
    """
    Stage 5: Link traffic sensor data to POA (postcode) polygons in NSW.
    Adds average traffic_count and distance to nearest station per POA.
    Handles both latitude/longitude and wgs84_latitude/wgs84_longitude naming.
    """
    print_header("Stage 5: Traffic Influence")

    # --- Load traffic dataset ---
    df_t = pd.read_csv(TRAFFIC_SENSORS_CSV)
    df_t.columns = df_t.columns.str.strip().str.lower()

    # --- Handle coordinate column variations ---
    if 'latitude' in df_t.columns and 'longitude' in df_t.columns:
        lat_col, lon_col = 'latitude', 'longitude'
    elif 'wgs84_latitude' in df_t.columns and 'wgs84_longitude' in df_t.columns:
        lat_col, lon_col = 'wgs84_latitude', 'wgs84_longitude'
    else:
        raise ValueError(
            "Traffic CSV must have either 'latitude'/'longitude' or 'wgs84_latitude'/'wgs84_longitude' columns."
        )

    if 'traffic_count' not in df_t.columns:
        raise ValueError("Traffic CSV missing required column: 'traffic_count'")

    # --- Build GeoDataFrame of traffic points ---
    gdf_t = gpd.GeoDataFrame(
        df_t,
        geometry=gpd.points_from_xy(df_t[lon_col], df_t[lat_col]),
        crs="EPSG:4326"
    ).to_crs(3857)

    # --- Prepare POA polygons ---
    gdf_p = gdf.copy()
    gdf_p = gdf_p[gdf_p.geometry.notnull() & ~gdf_p.geometry.is_empty]
    gdf_p = gdf_p.to_crs(3857)
    gdf_p['centroid'] = gdf_p.geometry.centroid

    print(f"✅ Loaded {len(gdf_t):,} traffic stations, linking to {len(gdf_p):,} POAs...")

    # --- Spatial join (nearest) ---
    gdf_joined = gpd.sjoin_nearest(
        gdf_p.set_geometry('centroid'),
        gdf_t[['traffic_count', 'geometry']],
        how='left',
        distance_col='dist_to_nearest_traffic_m'
    )

    # --- Aggregate average traffic per POA ---
    df_poa_traffic = (
        gdf_joined.groupby('POA_CODE', as_index=False)
        .agg({'traffic_count': 'mean', 'dist_to_nearest_traffic_m': 'mean'})
    )

    # Merge back
    gdf = gdf.merge(df_poa_traffic, on='POA_CODE', how='left')

    # Convert distance to km
    gdf['dist_to_nearest_traffic_km'] = (gdf['dist_to_nearest_traffic_m'] / 1000).round(3)
    gdf.drop(columns=['dist_to_nearest_traffic_m'], inplace=True)

    # Save output
    out_csv = os.path.join(OUT_DIR, "05_poa_traffic_summary.csv")
    gdf.drop(columns='geometry').to_csv(out_csv, index=False)
    print(f"💾 Traffic influence summary saved: {out_csv}")

    # Diagnostics
    missing = gdf['traffic_count'].isna().sum()
    print(f"🚧 POAs missing traffic data: {missing} / {len(gdf)}")

    return gdf

# ======================
# STAGE 6: DEMOGRAPHICS
# ======================
def stage_6_demographics(gdf):
    """
    Stage 6: Add demographic data (population + car commuters) to POA dataset.
    Ensures NSW-only filtering and safe handling of non-numeric POA codes.
    """

    print_header("Stage 6: Demographics")

    # --- Ensure POA_CODE exists and normalize ---
    if 'POA_CODE21' in gdf.columns:
        gdf = gdf.rename(columns={'POA_CODE21': 'POA_CODE'})
    gdf['POA_CODE'] = gdf['POA_CODE'].astype(str).str.zfill(4)

    # --- Safely convert POA_CODE to numeric (skip values like 'ZZZZ') ---
    gdf['POA_CODE_NUM'] = pd.to_numeric(gdf['POA_CODE'], errors='coerce')
    gdf = gdf.dropna(subset=['POA_CODE_NUM']).copy()
    gdf['POA_CODE_NUM'] = gdf['POA_CODE_NUM'].astype(int)

    # --- Filter to NSW only ---
    NSW_POSTCODE_RANGES = [(1000, 2599), (2620, 2899), (2921, 2999)]
    mask = False
    for lo, hi in NSW_POSTCODE_RANGES:
        mask |= gdf['POA_CODE_NUM'].between(lo, hi)

    gdf = gdf[mask].copy()
    print(f"✅ Retained {len(gdf):,} NSW POAs after filtering")

    # Drop helper column
    gdf = gdf.drop(columns=['POA_CODE_NUM'])

    # --------------------------------------------------------
    # Load ABS 2021 Population Data (POA-level)
    # --------------------------------------------------------
    df_pop = pd.read_csv("2021Census_G04B_NSW_POA.csv")
    df_pop['POA_CODE_2021'] = df_pop['POA_CODE_2021'].str.replace("POA", "").astype(int)

    df_pop = df_pop.rename(columns={'POA_CODE_2021': 'POA_CODE', 'Tot_P': 'population'})
    df_pop = df_pop[['POA_CODE', 'population']]

    # Merge population
    gdf['POA_CODE'] = gdf['POA_CODE'].astype(int)
    df_pop['POA_CODE'] = df_pop['POA_CODE'].astype(int)
    gdf = gdf.merge(df_pop, on='POA_CODE', how='left')

    # --------------------------------------------------------
    # Load ABS 2021 Travel-to-Work Data
    # --------------------------------------------------------
    df_travel = pd.read_csv("2021Census_G62_NSW_POA.csv")
    df_travel['POA_CODE_2021'] = df_travel['POA_CODE_2021'].str.replace("POA", "").astype(int)

    # sum commuters using car
    df_travel['car_commuters'] = (
        df_travel[['One_method_Car_as_driver_P',
                   'Two_methods_Trn_Car_as_drvr_P',
                   'Two_methods_Bus_Car_as_drvr_P']]
        .sum(axis=1)
    )

    df_travel = df_travel.rename(columns={'POA_CODE_2021': 'POA_CODE'})
    df_travel = df_travel[['POA_CODE', 'car_commuters']]
    df_travel['POA_CODE'] = df_travel['POA_CODE'].astype(int)

    # Merge travel data
    gdf = gdf.merge(df_travel, on='POA_CODE', how='left')

    # Fill missing values
    gdf['population'] = gdf['population'].fillna(0).astype(int)
    gdf['car_commuters'] = gdf['car_commuters'].fillna(0).astype(int)

    # Save output
    out_path = os.path.join(OUT_DIR, "06_poa_demographics_summary.csv")
    gdf.to_csv(out_path, index=False)
    print(f"💾 Saved enriched POA demographic file → {out_path}")

    show_df_diag(gdf.drop(columns='geometry', errors='ignore'), "POA + Demographics")

    return gdf

# ======================
# RUN ALL STAGES
# ======================
def run_all_stages():
    print_header("Starting Full Pipeline")

    d1 = stage_1_ev_demand()
    d2, active = stage_2_charger_supply()
    gdf = stage_3_merge_poa(d1, d2)
    gdf = stage_4_nearby_chargers(gdf, active)
    gdf = stage_5_traffic(gdf)
    gdf = stage_6_demographics(gdf)

    print_header("✅ Pipeline Completed Successfully")
    return gdf, active

if __name__ == "__main__":
    run_all_stages()


====== Starting Full Pipeline ======

====== Stage 1: EV Registration (Demand) ======

====== Saved: 01_ev_demand.csv ======
pcode  total_vehicles  ev_count  ev_proportion
 -000        22305347         0       0.000000
 0800            7813        40       0.005120
 0801               3         0       0.000000
 0804               3         0       0.000000
 0810           24447       164       0.006708

Shape: (2810, 4)


====== Stage 2: Charger Supply ======

====== Saved: 02_chargers_agg.csv ======
pcode  total_plugs  total_stations
 0nan            2             1.0
 2000          129           103.0
 2007            7             7.0
 2008            8             8.0
 2009            7             6.0

Shape: (374, 3)


====== STAGE 3 — Merge demand & supply into POA polygons ======

====== POA merged attributes (saved) ======
POA_CODE POA_NAME21 AUS_CODE21 AUS_NAME21  AREASQKM21                                         LOCI_URI21  SHAPE_Leng  SHAPE_Area pcode  total_vehicles  ev